In [1]:
import random
import time
import tkinter as tk
from tkinter import PhotoImage
from PIL import Image, ImageTk
import os

#------------------------------------------------------------------------------------------------------------

# Constants
BOARD_SIZE = 3
PLAYER = 'X'
AI = 'O'

#------------------------------------------------------------------------------------------------------------

# Add the trial_results list
trial_results = {
    "Olivia": [],
    "Emma": [],
    "Chris": [],
}

#------------------------------------------------------------------------------------------------------------


# Functions
def print_board(board):
    for row in board:
        print('|'.join(row))

def make_move(row, col, player):
    board[row][col] = player

    
def check_for_winner(player):
    for row in board:
        if row.count(player) == BOARD_SIZE:
            return True
    for col in range(BOARD_SIZE):
        if board[0][col] == player and board[1][col] == player and board[2][col] == player:
            return True

    if board[0][0] == player and board[1][1] == player and board[2][2] == player:
        return True

    if board[0][2] == player and board[1][1] == player and board[2][0] == player:
        return True

    return False

#------------------------------------------------------------------------------------------------------------


def check_for_tie():
    for row in board:
        if ' ' in row:
            return False
    return True

#------------------------------------------------------------------------------------------------------------


def minimax(is_maximizing, depth):
    result = check_game_over()
    if result[0]:
        if result[1] == AI:
            return 100 - depth
        elif result[1] == PLAYER:
            return -100 + depth
        else:
            return 0

    if is_maximizing:
        best_score = float('-inf')
        for row in range(BOARD_SIZE):
            for col in range(BOARD_SIZE):
                if board[row][col] == ' ':
                    board[row][col] = AI
                    score = minimax(False, depth + 1)
                    board[row][col] = ' '
                    best_score = max(best_score, score)
        return best_score
    else:
        best_score = float('inf')
        for row in range(BOARD_SIZE):
            for col in range(BOARD_SIZE):
                if board[row][col] == ' ':
                    board[row][col] = PLAYER
                    score = minimax(True, depth + 1)
                    board[row][col] = ' '
                    best_score = min(best_score, score)
        return best_score
import random

#------------------------------------------------------------------------------------------------------------

def medium_ai_move():
    optimal_move_probability = 0.7

    if random.random() < optimal_move_probability:
        best_score = float('-inf')
        for row in range(BOARD_SIZE):
            for col in range(BOARD_SIZE):
                if board[row][col] == ' ':
                    board[row][col] = AI
                    score = minimax(False, 0)
                    board[row][col] = ' '
                    if score > best_score:
                        best_score = score
                        best_move = (row, col)
        row, col = best_move
    else:
        while True:
            row = random.randint(0, BOARD_SIZE-1)
            col = random.randint(0, BOARD_SIZE-1)
            if board[row][col] == ' ':
                break

    return row, col
   
#------------------------------------------------------------------------------------------------------------
    
def clear_board():
    global board
    
    board = [[' ', ' ', ' '],
             [' ', ' ', ' '],
             [' ', ' ', ' ']]
    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            buttons[row][col].config(state='normal', text=' ')
    result_label.config(text='')

def start_game():
    clear_board()
    result_label.config(text='')
    
    
def reset_game():
    global board, trial_results
    
    # Get the name and grade inputs
    name = first_name_entry.get()
    grade = grade_entry.get()
    
    # Add the name and grade to the trial_results dictionary
    if name and grade:
        opponent_name = ""
        if difficulty_var.get() == "easy":
            opponent_name = "Olivia"
        elif difficulty_var.get() == "medium":
            opponent_name = "Emma"
        else:
            opponent_name = "Chris"
        trial_results[opponent_name].append(f"{name} ({grade}): {result_label.cget('text')}")

    board = [[' ', ' ', ' '],
             [' ', ' ', ' '],
             [' ', ' ', ' ']]
    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            buttons[row][col].config(state='normal', text=' ')
    result_label.config(text='')

#------------------------------------------------------------------------------------------------------------

    
def display_trial_results():
    global trial_results
    
    # Get the path to your desktop
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    
    # Create the file path for the trial results text file
    file_path = os.path.join(desktop_path, "trial_results.txt")
    
    # Write the trial results to the text file
    with open(file_path, 'a') as file:
        for opponent_name, results in trial_results.items():
            file.write(f"Results against {opponent_name}:\n")
            for index, result_text in enumerate(results):
                file.write(f"Game {index+1}: {result_text}\n")
            file.write("---------------------------------\n")
    
    # Display the trial results in a new window
    result_window = tk.Toplevel(root)
    result_window.title("All Trial Results")

    for opponent_name, results in trial_results.items():
        opponent_label = tk.Label(result_window, text=f"Results against {opponent_name}:", font=('Arial', 16, 'bold'))
        opponent_label.pack(padx=20, pady=10)

        for index, result_text in enumerate(results):
            trial_result_label = tk.Label(result_window, text=f"Game {index+1}: {result_text}", font=('Arial', 14))
            trial_result_label.pack(padx=20, pady=5)
    
    # Clear the trial results and close the result window
    trial_results = {
        "Olivia": [],
        "Emma": [],
        "Chris": [],
    }
    result_window.protocol("WM_DELETE_WINDOW", result_window.destroy)
    # Clear the name and grade boxes
    first_name_entry.delete(0, 'end')
    grade_entry.delete(0, 'end')

#------------------------------------------------------------------------------------------------------------

    
def on_button_click(row, col, difficulty):
    if board[row][col] == ' ':
        # Make the player's move
        buttons[row][col].config(text=PLAYER, state='disabled', fg='blue', font=('Arial', 60, 'bold'))

        make_move(row, col, PLAYER)

        # Check if the game is over
        if check_for_winner(PLAYER):
            print_board(board)
            result_label.config(text="Great job, you're the Champion!")
            return
        elif check_for_tie():
            print_board(board)
            result_label.config(text="Wow, What a Close Game!")
            return

        # Make the AI's move
        root.after(1000, ai_move, difficulty, None)

#------------------------------------------------------------------------------------------------------------        
        
def check_game_over():
    # Check rows for win
    for row in board:
        if all([cell == 'X' for cell in row]):
            return True, 'X'
        elif all([cell == 'O' for cell in row]):
            return True, 'O'
    # Check columns for win
    for col in range(BOARD_SIZE):
        if all([board[row][col] == 'X' for row in range(BOARD_SIZE)]):
            return True, 'X'
        elif all([board[row][col] == 'O' for row in range(BOARD_SIZE)]):
            return True, 'O'
    # Check diagonals for win
    if all([board[i][i] == 'X' for i in range(BOARD_SIZE)]):
        return True, 'X'
    elif all([board[i][i] == 'O' for i in range(BOARD_SIZE)]):
        return True, 'O'
    if all([board[i][BOARD_SIZE - i - 1] == 'X' for i in range(BOARD_SIZE)]):
        return True, 'X'
    elif all([board[i][BOARD_SIZE-i-1] == 'O' for i in range(BOARD_SIZE)]):
        return True, 'O'
    # Check for tie
    if check_for_tie():
        return True, 'Tie'
    # Game is not over
    return False, None

#------------------------------------------------------------------------------------------------------------

def ai_move(difficulty, event):
    if difficulty == "easy":
        while True:
            row = random.randint(0, BOARD_SIZE-1)
            col = random.randint(0, BOARD_SIZE-1)
            if board[row][col] == ' ':
                break
    elif difficulty == "medium":
        row, col = medium_ai_move()
    else:
        best_score = float('-inf')
        for row in range(BOARD_SIZE):
            for col in range(BOARD_SIZE):
                if board[row][col] == ' ':
                    board[row][col] = AI
                    score = minimax(False, 0)
                    board[row][col] = ' '
                    if score > best_score:
                        best_score = score
                        best_move = (row, col)
        row, col = best_move

    # Make the move
    buttons[row][col].config(state='disabled', text=AI, fg='blue', font=('Arial', 60, 'bold'))
    make_move(row, col, AI)

    if check_for_winner(AI):
        print_board(board)
        result_label.config(text="Better Luck Next Time!")
        return
    elif check_for_tie():
        print_board(board)
        result_label.config(text="Wow, What a Close Game!")
        return

#--------------------------------------------------------------------------------------------------------------    

    
root = tk.Tk()
root.title("UML-Tic Tac Toe AI Game")

background_image = Image.open("logo.png")
width, height = root.winfo_screenwidth(), root.winfo_screenheight()
background_image = background_image.resize((width, height), Image.ANTIALIAS)

background_photo = ImageTk.PhotoImage(background_image)
background_label = tk.Label(root, image=background_photo)
background_label.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

#------------------------------------------------------------------------------------------------------------


# Get the screen resolution
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

# Set the dimensions of the root window to fill the screen
root.geometry(f"{screen_width}x{screen_height}")


profile_frame = tk.Frame(root, bg='gray')
profile_frame.grid(row=1, column=BOARD_SIZE+1, padx=10, pady=10)

first_name_label = tk.Label(profile_frame, text="First Name:", font=('Arial', 15))
first_name_label.grid(row=0, column=0, sticky=tk.E, pady=5)

first_name_entry = tk.Entry(profile_frame, width=15, font=('Arial', 14), highlightthickness=2)
first_name_entry.grid(row=0, column=1, pady=5)

grade_label = tk.Label(profile_frame, text="Grade:      ", font=('Arial', 15))
grade_label.grid(row=1, column=0, sticky=tk.E, pady=10)

grade_entry = tk.Entry(profile_frame, width=15, font=('Arial', 14), highlightthickness=2)
grade_entry.grid(row=1, column=1, pady=10)

reset_button = tk.Button(profile_frame, text="Reset", font=('Arial', 16), command=reset_game, width=6, height=1)
reset_button.grid(row=2, column=0, pady=10, padx=10, sticky="W")

start_button = tk.Button(profile_frame, text="New Start", font=('Arial', 16), command=start_game, width=9, height=1)
start_button.grid(row=2, column=1, pady=10, padx=10, sticky="E")



player_profile = tk.Label(profile_frame, text="", font=('Arial', 16))
#------------------------------------------------------------------------------------------------------------


# Add a frame for the game board
board_frame = tk.Frame(root, bg='black')

# Add the game board buttons
buttons = []
for row in range(BOARD_SIZE):
    button_row = []
    for col in range(BOARD_SIZE):
        button = tk.Button(board_frame, text=' ', font=('Arial', 60), width=3, height=1,
                           command=lambda row=row, col=col: on_button_click(row, col, difficulty_var.get()))
        button.grid(row=row, column=col, padx=10, pady=10)
        button_row.append(button)
    buttons.append(button_row)

# Center the board frame in the window
board_frame.place(relx=0.7, rely=0.5, anchor=tk.CENTER)


result_label = tk.Label(root, font=('Arial', 20), pady=10)
result_label.place(relx=0.7, rely=0.05, anchor=tk.CENTER)

#------------------------------------------------------------------------------------------------------------


# Add a frame for buttons
button_frame = tk.Frame(root)
button_frame.grid(row=0, column=BOARD_SIZE+1, rowspan=BOARD_SIZE+2, padx=10, pady=10)


# Add radio buttons for difficulty level
difficulty_label = tk.Label(button_frame, text="Choose an AI Opponent:", font=('Arial', 16))
difficulty_label.grid(row=1, column=0, pady=10, sticky=tk.W)

difficulty_var = tk.StringVar()
difficulty_var.set("easy")

easy_rb = tk.Radiobutton(button_frame, text="Olivia ", variable=difficulty_var, value="easy", height=2, width=15 , font=('Arial', 14))
easy_rb.grid(row=2, column=0, padx=5, pady=10, sticky=tk.W)

medium_rb = tk.Radiobutton(button_frame, text="Emma", variable=difficulty_var, value="medium", height=2, width=15, font=('Arial', 14))
medium_rb.grid(row=3, column=0, padx=5, pady=10, sticky=tk.W)

hard_rb = tk.Radiobutton(button_frame, text="Chris", variable=difficulty_var, value="hard", height=2, width=15, font=('Arial', 14))
hard_rb.grid(row=4, column=0, padx=5, pady=10, sticky=tk.W)
#------------------------------------------------------------------------------------------------------------


# Center the button frame in the window
button_frame.configure(bg='gray')
button_frame.place(relx=0.05, rely=0.6, anchor=tk.W)

# Add the show_all_results_button code here
show_all_results_button = tk.Button(button_frame, text="Show Results", font=('Arial', 16), command=display_trial_results, width=15, height=1)
show_all_results_button.grid(row=7, column=0, pady=5, padx=5, sticky="W")


#Set up the game board
board = [[' ', ' ', ' '],
[' ', ' ', ' '],
[' ', ' ', ' ']]

#Start the main event loop
root.mainloop()